In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

TOKEN = os.environ["INVEST_TOKEN"]

In [3]:
from datetime import timedelta
from pandas import DataFrame
from tinkoff.invest import CandleInterval, Client
from tinkoff.invest.utils import now

all_candles = []

def main():

    with Client(TOKEN) as client:
        for candle in client.get_all_candles(
            figi="BBG004730N88",    # FIGI Composite of SBER
            from_=now() - timedelta(days=90*2),
            interval=CandleInterval.CANDLE_INTERVAL_DAY,
        ):
            all_candles.append(
                {
                    "Time": candle.time,
                    "Open": candle.open,
                    "Close": candle.close,
                    "Max": candle.high,
                    "Min": candle.low,
                    "Volume": candle.volume,
                }
            )

    return 0


if __name__ == "__main__":
    main()


In [4]:
from modules.modules import Quotation, factor
def convert_quotation(q: Quotation):
    return q.units + q.nano / factor

In [5]:
df = DataFrame(all_candles)
for column in ["Open", "Close", "Max", "Min"]:
    df[column] = df[column].apply(convert_quotation)

df.set_index("Time", inplace=True)
# df

In [6]:
from modules.modules import Window

wi = Window(small=9, medium=26, large=52)

In [7]:
df["Tenkan-sen"]=(df["Max"].rolling(window=wi.small).max() + df["Min"].rolling(window=wi.small).min()) / 2
df['Kijun-sen'] = (df['Max'].rolling(window=wi.medium).max() + df['Min'].rolling(window=wi.medium).min()) / 2
df['Senkou Span A'] = ((df['Tenkan-sen'] + df['Kijun-sen']) / 2).shift(wi.medium)
df['Senkou Span B'] = ((df['Max'].rolling(window=wi.large).max() + df['Min'].rolling(window=wi.large).min()) / 2).shift(wi.medium)
df['Chikou Span'] = df['Close'].shift(-wi.medium)

In [49]:
df.head()

,Open,Close,Max,Min,Volume,Tenkan-sen,Kijun-sen,Senkou Span A,Senkou Span B,Chikou Span
Time,,,,,,,,,,
2024-05-28 00:00:00+00:00,317.50,318.22,320.90,315.82,3018779,NaN,NaN,NaN,NaN,324.66
2024-05-29 00:00:00+00:00,318.25,320.38,320.40,315.92,2298926,NaN,NaN,NaN,NaN,325.00
2024-05-30 00:00:00+00:00,320.91,316.63,321.55,316.00,2700539,NaN,NaN,NaN,NaN,324.07
2024-05-31 00:00:00+00:00,316.00,313.11,318.49,309.80,4920645,NaN,NaN,NaN,NaN,319.99
2024-06-03 00:00:00+00:00,313.50,310.95,315.50,305.00,6186077,NaN,NaN,NaN,NaN,316.68


In [8]:
import plotly.graph_objects as go

fig = go.Figure(data=[go.Candlestick(x=df.index,
                open=df['Open'],
                high=df['Max'],
                low=df['Min'],
                close=df['Close'],
                increasing_line_color='black',
                decreasing_line_color='grey')])

fig.add_trace(go.Scatter(x=df.index, y=df['Tenkan-sen'], mode='lines', name='Tenkan-sen', line=dict(color='red')))
fig.add_trace(go.Scatter(x=df.index, y=df['Kijun-sen'], mode='lines', name='Kijun-sen', line=dict(color='blue')))

fig.add_trace(go.Scatter(x=df.index, y=df['Senkou Span A'], mode='lines', name='Senkou Span A', line=dict(color='yellow')))
fig.add_trace(go.Scatter(x=df.index, y=df['Senkou Span B'], mode='lines', name='Senkou Span B', line=dict(color='purple')))

fig.add_trace(go.Scatter(x=df.index, y=df['Senkou Span A'], fill='tonexty', fillcolor='rgba(0,100,80,0.2)', line=dict(color='rgba(255,257,255,0)'), name='Kumo'))

fig.add_trace(go.Scatter(x=df.index, y=df['Chikou Span'], mode='lines', name='Chikou Span', line=dict(color='green')))

buySignals = (df['Tenkan-sen'] > df['Kijun-sen']) & (df['Tenkan-sen'].shift(1) <= df['Kijun-sen'].shift(1))
sellSignals = (df['Tenkan-sen'] < df['Kijun-sen']) & (df['Tenkan-sen'].shift(1) >= df['Kijun-sen'].shift(1))

fig.add_trace(go.Scatter(x=df.index[buySignals], y=df['Min'][buySignals], mode='markers', marker=dict(color='green', size=8), name='Buy Signal'))
fig.add_trace(go.Scatter(x=df.index[sellSignals], y=df['Max'][sellSignals], mode='markers', marker=dict(color='red', size=8), name='Sell Signal'))

fig.show()